## **Install libraries**

In [ ]:
!pip install dash
!pip install jupyter_dash
!pip install dash_gif_component
!pip install dash_bootstrap_components
!pip install dash_daq

## **Import libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.express as px
import plotly.io as pio
import dash
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import html, dcc, Dash, Input, Output, State
import numpy as np
import time
import dash_gif_component as gif
import seaborn as sns
import plotly.graph_objects as go

#model libraries
from sklearn.preprocessing import LabelEncoder , StandardScaler ,PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import pickle


## **Reading Data**

In [ ]:
df=pd.read_csv("/content/heart.csv")

## **Data Preprocessing**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


**Check Duplicated**

In [ ]:
df.isna().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

**Categorecal Features**

In [ ]:
dup=df.duplicated()
dup=df[dup]
dup

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [ ]:

categorical_cols= df.select_dtypes(include=['object'])
categorical_cols.columns

Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object')

In [ ]:
labels = [x for x in df.ChestPainType.value_counts().index]
values = df.ChestPainType.value_counts()
labels

['ASY', 'NAP', 'ATA', 'TA']

## **Data Visualization**

In [ ]:
data=df.copy()

In [ ]:
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])

fig.update_layout(
    title_text="Distribution of data by Chest Pain Type (in %)")

fig.update_traces()
fig.show()


In [ ]:
fig=px.histogram(df,
                 x="HeartDisease",
                 color="Sex",
                 hover_data=df.columns,
                 title="Distribution of Heart Diseases by Gender",
                 barmode="group")
fig.update_xaxes(tickvals=[0, 1], ticktext=['Normal', 'Heart disease'])

fig.show()

In [ ]:
fig_chest_sex = px.histogram(df, x='Sex', color='ChestPainType', facet_col='HeartDisease', barmode='group')

fig_chest_sex.show()

In [ ]:
x = df.groupby("Age")['RestingBP'].mean().index
y = df.groupby("Age")['RestingBP'].mean().values

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='lines',
    line=dict(color='rgb(0,100,80)'),
    name='Blood Pressure'
))

fig.show()

In [ ]:
x = df.groupby("Age")['Oldpeak'].mean().index
y = df.groupby("Age")['Oldpeak'].mean().values

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='lines',
    line=dict(color='rgb(0,100,80)'),
    name='Blood Pressure'
))

fig.show()

In [ ]:
data['Sex'] = np.where(data['Sex'] == "M", 0, 1)
data['ExerciseAngina'] = np.where(data['ExerciseAngina'] == "N", 0, 1)

In [ ]:
agg_df = data.groupby('HeartDisease')['ExerciseAngina'].count().reset_index()

fig = px.bar(agg_df, x='HeartDisease', y='ExerciseAngina')
fig.show()

In [ ]:
agg_df = data.groupby('Sex')['ExerciseAngina'].count().reset_index()

fig = px.bar(agg_df, x='Sex', y='ExerciseAngina')
fig.update_xaxes(tickvals=[0, 1], ticktext=['Female', 'Male'])

fig.show()

In [ ]:
agg_df = data.groupby('Sex')['FastingBS'].count().reset_index()

fig = px.bar(agg_df, x='Sex', y='FastingBS')
fig.update_xaxes(tickvals=[0, 1], ticktext=['Female', 'Male'])

fig.show()

In [ ]:
correlation_all = df.apply(lambda x : pd.factorize(x)[0]).corr(method = 'pearson')
df_corr = correlation_all
df_corr_negative = df_corr[df_corr['HeartDisease'] < 0] .copy()
df_corr_negative*=-1
df_corr[df_corr['HeartDisease'] < 0]  = df_corr_negative
df_corr = df_corr.sort_values(by='HeartDisease' , ascending=False)
df_corr.drop('HeartDisease',axis = 0,inplace = True)
df_corr = df_corr.rename_axis('Features')
fig = px.bar(df_corr ,
       x = 'HeartDisease' ,

       labels={'HeartDisease':'Effect on Results'},
       title = 'Each feature effect on final result'
      )
fig.show()

# **Model**

In [ ]:
cate_cols=['ChestPainType', 'RestingECG', 'ST_Slope']

In [ ]:
encoder = LabelEncoder()

for col in cate_cols:
    data[col] = encoder.fit_transform(data[col])


In [ ]:
data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,1,140,289,0,1,172,0,0.0,2,0
1,49,1,2,160,180,0,1,156,0,1.0,1,1
2,37,0,1,130,283,0,2,98,0,0.0,2,0
3,48,1,0,138,214,0,1,108,1,1.5,1,1
4,54,0,2,150,195,0,1,122,0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,0,3,110,264,0,1,132,0,1.2,1,1
914,68,0,0,144,193,1,1,141,0,3.4,1,1
915,57,0,0,130,131,0,1,115,1,1.2,1,1
916,57,1,1,130,236,0,0,174,0,0.0,1,1


In [ ]:
x_newdata=data.drop(['HeartDisease'],axis=1)
y_newdata=data['HeartDisease']


In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(x_newdata, y_newdata, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC()
}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Model Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\n{model}\nAccuracy: {accuracy:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}\nF1 Score: {f1:.4f}\n")

Training Logistic Regression...

LogisticRegression()
Accuracy: 0.8696
Precision: 0.8777
Recall: 0.8696
F1 Score: 0.8707

Training Decision Tree...

DecisionTreeClassifier()
Accuracy: 0.7391
Precision: 0.7597
Recall: 0.7391
F1 Score: 0.7418

Training Random Forest...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression




RandomForestClassifier()
Accuracy: 0.8696
Precision: 0.8724
Recall: 0.8696
F1 Score: 0.8702

Training Support Vector Machine...

SVC()
Accuracy: 0.6884
Precision: 0.6999
Recall: 0.6884
F1 Score: 0.6913



In [ ]:
model=LogisticRegression()
model.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [ ]:
X_train.shape

(642, 11)

In [ ]:
lrpred = model.predict(X_test)

## **Save Model**

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(round(result,2)*100)

87.0


# **DashBoard**

In [ ]:
df2 = df.copy()
def x(c):
    if c == 0:
        return 'no'
    else:
        return 'yes'
df2['HeartDisease'] = df2['HeartDisease'].apply(x)
df2[df2['Sex']=='M' ]
df2

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,no
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,yes
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,no
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,yes
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,no
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,yes
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,yes
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,yes
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,yes


In [ ]:
x_df2=df2.drop(['HeartDisease'],axis=1)
y_df2=df2['HeartDisease']

In [ ]:
XX_train, XX_temp, yy_train, yy_temp = train_test_split(x_df2, y_df2, test_size=0.3, random_state=42)
XX_val, XX_test, yy_val, yy_test = train_test_split(XX_temp, yy_temp, test_size=0.5, random_state=42)

In [ ]:
df2

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,no
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,yes
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,no
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,yes
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,no
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,yes
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,yes
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,yes
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,yes


In [ ]:
XX_train.shape

(642, 11)

In [ ]:
XX_test.shape

(138, 11)

In [ ]:
df_temp = df2.copy()
background_color = '#f1f1f1'
number_of_men = df_temp[df_temp['Sex'] == "M"].count()[0]
number_of_women = df_temp[df_temp['Sex'] == "F"].count()[0]
number_of_patients = df_temp[df_temp['HeartDisease'] == 'yes'].count()[0]
number_of_non_patients = df_temp[df_temp['HeartDisease'] == 'no'].count()[0]

**Make Predection**

In [ ]:
def make_prediction(n, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11):

    # Check if the button is clicked
    if n is None or n == 0:
        return "Not clicked.", ""
    else:
        n = n + 1
        # Assuming you have the necessary preprocessing steps before making predictions
        features_list = pd.DataFrame([{'Age': x1, 'Sex': x2, 'ChestPainType': x3, 'RestingBP': x4,
                                       'Cholesterol': x5, 'FastingBS': x6, 'RestingECG': x7, 'MaxHR': x8,
                                       'ExerciseAngina': x9, 'Oldpeak': x10, 'ST_Slope': x11}])

        # Assuming you have the necessary preprocessing steps before making predictions
         #1
        features_list['Sex'] = np.where(features_list['Sex'] == "Male", 0, 1)
        features_list['ExerciseAngina'] = np.where(features_list['ExerciseAngina'] == "No", 0, 1)
        features_list['FastingBS'] = np.where(features_list['FastingBS'] == "more than 120 mg/dl", 1, 0)
        features_list["ChestPainType"] = features_list["ChestPainType"].replace({"ATA": 1, "NAB": 2, "ASY": 0,"TA":3})
        features_list["RestingECG"] = features_list["RestingECG"].replace({"Normal": 1, "ST": 2, "LVH": 0})
        features_list["ST_Slope"] = features_list["ST_Slope"].replace({"Up": 2, "Flat": 1, "Down": 0})

        with open("/content/finalized_model.sav", "rb") as file:
            loaded_model = pickle.load(file)

        input_features = features_list.values.tolist()

        pred_res = loaded_model.predict(input_features)
        prop_res = loaded_model.predict_proba(input_features)

        if pred_res[0] == 0:
            diagnose_detect = 'Negative'
            diagnose_prob = prop_res[0][0]
        elif pred_res[0] == 1:
            diagnose_detect = 'Positive'
            diagnose_prob = prop_res[0][1]

        return f'The Prediction  --->{diagnose_detect}' ,f'The Probability ---> {round(diagnose_prob,2)*100}%'

## **Dash**

In [ ]:
import dash_daq as daq
app =JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])

/usr/local/lib/python3.10/dist-packages/dash/dash.py:550: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [ ]:
age_range = df2['Age'].unique()
slider = html.Div(children =[
        html.Center(html.B('age of individuals') ),
        html.Div(dcc.RangeSlider(
                id='age_range',
                min = age_range.min(),
                max = age_range.max(),
                step = 1 , # to make the slider not to stop in the middle
                allowCross=False,
                marks = {int(age):str(age) for age in age_range if age%10 ==0 },
                value = [30,35] ,
                vertical = False ,
                verticalHeight = 600
                )#end slider
                  )  #end div
     ])
#######################
## drop down sex ######
#######################

drop_down_sex = html.Div([
                        html.Center(html.B('sex of individuals')),
                        html.Div(dcc.Dropdown(
                            id= 'drop_down_sex',
                            options={
                            'Male': 'male',
                            'Female': 'female',
                               },
                               value=['Male','Female'],
                                multi=True ,
                                style={'background-color': background_color , 'border-color':'blue','height':'60%' , 'width':'75%' , 'margin-left':'15%'}
                        )#end drop down
                                )

                    ] ) # end div

#######################
## title ##############
#######################

title = html.Div(html.H1('Heart disease failure data', style = {'color' : 'red' , 'margin-top':'10px'}) )

##############
## man card ##
##############

card_man = dbc.Card(
    id= 'card_men',
    children=[
    dbc.CardBody( [
                        html.Div([
                            html.P(['Number of MEN']),
                            html.Center([
                                html.H3(number_of_men,id = 'number_of_men' ,style={'margin-top':'-10px'})# this negative is working (20 is large)
                            ]),
                        ],className='card-text'),
    ]) ,],
                        style={"height": 100 , 'width':'17rem', "background-color": "white" , 'border-color' : 'yellow' ,'border-width': '0px 0px 3px 0px',}
    ,
)#card


##################
## woman card ###
###################
card_woman = dbc.Card(
    id='card_women',
    children=[
    dbc.CardBody( [
                        html.Div([
                            html.P(['Number of WOMEN']),
                            html.Center([
                                html.H3(number_of_women, id= 'number_of_women' , style={'margin-top':'-10px'})
                            ]),

    ], className='card-text') ,
    ]),],
                        style={"height": 100 , 'width':'17rem', "background-color": "white",'border-width': '0px 0px 3px 0px' , 'border-color' : 'green' ,}
    ,
)#card


##################
##patient card ##
##################
card_patient = dbc.Card(
    id= 'card_patient',
    children=[
    dbc.CardBody( [
                        html.Div([
                            html.P(['Number of PATIENTS']),
                            html.Center([
                                html.H3(number_of_patients , id= 'number_of_patients' , style={'margin-top':'-10px'})
                            ]),

    ], className='card-text') ,
    ]) , ],
                        style={"height": 100 , 'width':'17rem', "background-color": "white", 'border-color' : 'red' ,'border-width': '0px 0px 3px 0px', }
    ,
)#card



#######################
## non patient card ##
#######################
card_non_patient = dbc.Card(
    id= 'card_non_patient',
    children=[
    dbc.CardBody( [
                        html.Div([
                            html.P(['Number of NON PATIENTS']),
                            html.Center([
                                html.H3(number_of_non_patients, id= 'number_of_non_patients' , style={'margin-top':'-10px'})
                            ]),

    ], className='card-text') ,
    ]) ,],
            style={"height": 100 , 'width':'17rem', "background-color": "white", 'border-color' : 'blue' ,'border-width': '0px 0px 3px 0px', }
    ,
)#card

In [ ]:
import base64
image_filename = '/content/image2.jpg'

app.layout = html.Div(children =[

    dcc.Tabs([
      dcc.Tab(label='Heart Failure Prediction Model',style={'background-color':'azure'}, children=[


      dbc.Alert(
        "<<<< Heart Failure Prediction >>>>", className="m-5",
        style={'text-align':'center','font-size':21,'background-color':'bluish'}),
            html.Div([
                html.Div([
                            html.Img(

                     src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAHwAfAMBEQACEQEDEQH/xAAcAAEAAQUBAQAAAAAAAAAAAAAABgIDBAUHAQj/xAA7EAABAwIEAwYEBAILAAAAAAABAAIDBBEFEiExBkFRBxMiYXGBFDKRwVKhsdEjMxU0QlNicoKywtLw/8QAGgEBAAIDAQAAAAAAAAAAAAAAAAIDAQQFBv/EAC4RAAICAQMDAQcDBQAAAAAAAAABAhEDBBIhBTFBURMiMmFxgcFCodEUIySR8P/aAAwDAQACEQMRAD8A7igCAIAgCAIDwuA3QGNW4jR0DWuramGna75TLIG39LrEpKK5ZZjw5MrrHFv6F2OeKSnbURyMdC5ucSB3hLd736JfFkXGSlta5Kaerp6kF1PNFKBuY3hwH0RNPsJQlD4lRXJNHE3NK9rG9XGwWTCTk6Ssqa4PALSCCLgg7oY7FSAIAgCAIAgCAIAgCAgnaDE74+kcXvyOjNm5jYEHe23MfRaepdNHf6NteOXHKZBsZnrcwiq5pJKVr/4edxIbcW06cvotdttcna00IrI3FU6/JNp8TZh3ZpTRZwaippvh4mA63NwfoL/RbTntwL1OJHTvN1WTriLt/bn9zWcJYtTYBgWKVmVhnfO2GGEGxkcGC3t4jdQxzUItm31HTT1epx4/FNt+nJYw3CMV4vnmqqisYRG7KZpmFzWu6RsuBpfr9ViMJZbbZbn1On6bFQjHv4XD+7Oj4Bhr8IwuGhfUGoMQIEhbl0JJtbkBstuEdsaPM6rP/UZpZKqzYqZrhAEAQBAEAQBAEAQEI47eH11JF/dxOe73It/tWlqu6R3ujprHOXq0c14grfiJhSxeINN3+Z6LVPTabHtW5lVEx7Iw6cNDrWzEaqceO5HI03UT3+D3+eOZjX+aUvBm5baa4Om9mNRF/QL6QuHfwTOLh1DjcH9R7Lc0z92jyvXIS/qfaeGl+xMVsHGCAIAgCAIAgCAIAgKXvbG0ue4BoFyTyQLng5Rxlib3VtVUNvd5DYdOWw+5XOzNudnrul4oPFGK+5G6OiZTxGWfxSO1uVFRrlnTyZXJ7Y9jClknr6ptNSgZibeJwaB6k6AeaJX3JuUcMdzR5iGDVtBG6aYQyRBwaZYX5gCduQ6HVZoji1ccklFqm/Uy+E8bmwXHaSpEh7kvEc7b/Mx2h+m/spQeyVkOoaWOp08oee6+q/nsfQC6B4AIAgCAIAgCAIAgMKtxKGk8Ju+XlG3f36KMpqJZDFKfY0VbWVFX/NIazkxuw/dUSk2bkMUYdiEcU2kroBuGku9wLfdUz8HZ6Y6c3/3c0mJVGSH7KuTs6+GHNlnhqPvDPNYm5ADrctSfzSXCI55XI3lfQ/0jTCKXMGjUFptYqCdFClGLvyamPhvuqhjp5nSQNcC+NoAc8cxfldT3oteebi1F8nXsI4qw3EY2jM+lkOnd1Ayn67LejmhLizyObp2oxc1a+X8dzetIcAQbg6q00T1AEAQBAEAQGHilYKOlLxbO45WA9SoylSLMcN8qIznNy9xzPcblx3JWvZ0EklSMSrrGRNN3XPkomSKYm91ZOBF8zXXc4bN0tbzVc6fY6vT4vG3KfnsvyURYbC9wMrBK4c36ge2yps6MsjSNtDDHG0WAQ1JTkzx04vliBef8OykotlcnGHxuig09VLqY/ZS9n8yC12OLpJmPXGWGlmaYckndkNIvdR2tPkvjkjljcX9ifcANmbwtRCfNfxZc2+XMbLfw/Aed6i4vUyr5EjVpohAEAQBAEBGeNKgQtpA4OLXF506i37qnL4NrTcNkOxDGW0sV7uudgTqVQ+FydHDhnmltgiH12O1dRKWhwYwXuBzt5qtybR2MOjxYnb5fzKcHxE087xP3j2PGuXXKetlFx4o2Gk3fkklLXQSi8Esb+rb6/TdQpohKNluTFaT4uOkfO0SPNg0a69CdgrMcL5Zqamcscfc7kloBAB4ALEaE/dXHHu+TZNLQFkGj4tjMmDVBicWPaM92mxsNVhm1opbdRG/Wv9m07KcWqsRwyqir60zywSgRse6744y0WueYJB3V2GTa5Zr9ZwRxZk4RpNfaydK45AQBAEAQBAariTDjiWGPjjF5mHPH5kcvcKMlaLMU9srOCV9d8bXvnDrxN+TXTKNlz5u+T2elxLDBR+7+phMD7ZmtJI1KeCwz8NDY7yEXB0Cw2ZLGJENd3lgLg7LKIyltVmHTU5mcXv8Ak29SpSe3hFGPF7R7pdjZMxmvoqjPRy+EDK4P8QdbrdIv1I6jBGaSUeES7AeLIa8iGcfD1PJpN2O9D9lNM5mXTuC3LlG8kYKxhik8bZBksDvdZKFLb7y8Eo4Y4Ww/hplQ2g75zp3AvkmcHOsNhoBoLrZhBQ7Gnqtbl1Uk8nj0N6pmoEAQBAEAQGu4kqDScPYnUtveGkleLeTCVGbqLZbp47ssYvy0fOMQHw7rb3Fh1A/8FzezPbRe6LkbGmex0Qa217ahZZkrjjbE0ht7brBgwakipe1oIFzYNsb/AKKStEZpS4ZdqAIDAyM3azxHoTdYXJMxpvA4huoOoPks2Yboxy7K8G9jdSXKNaT2zs6n2VTx4vPN8ZI41VHle1vJ4Ozj5gj9Fdge7ucXqeL2NOHZnUhsto44QBAEAQBAEBgY9SmtwPEKRou6emkjHqWkKMlaaJ4pbZqXoz5pikzRtcNiLj3C5x7KE/BkN2CkXlRc4ixc4j1SgUiTumueD4vlaehKwRlJRW4Rk/DWcdWv09CD+yeeDEG65KHG+lvMIyT9CwRmcQeQUk6RqSjum7J72Md4eJ6k3OUUT83mc7LfdW4fj4Ob1OX+Mk+9/hnaltnBCAIAgCAIAgBQHzhxdhbsE4mxCgylsQlMsGmhjebtt6at9lo5Y7ZHptHm9piT8o10btLc1BHThK0XBYi5vvaw5oTLM5/lAAAanRI9mU5v0ovNyiAh2neOuD0tzUWWLmP1LLw5rrO0IQi2138Fs/Nfy1UvBrSdys6n2H4eTT4lizgQyVzaeK4/DcuI8ruaP9K2sEatnE6nkTlGK8HU1ecsIAgCAIAgCAIDnva7w1LimGR4tQRmSsoAc8bRrJCfmA8xv9VVlhuRvaHUeynT7M43FI17Q9huDstOqPRQlzaLveAC6Jcl7yIoddzYyTrqEvuVyuSi2XZXWeW/hGUKJc/iotlxdG0nceH9lmuSuTuFlqCCfEa2DD6BveVNQ8RsaOZP26+SnGNs08uSMItvwfS3DmDw4DgtJhlNqyCPKXfjdu53ublb0VSo8zlyPJNyfk2SyQCAIAgCAIAgCAEIDivadwJLhc8uO4FDeiec9VTsH8k83gfh69PTajJj8o62j1n6JHPYphLYD5ugWvVHV3WZF9AoGyuyPZSTI71RGcknudFiolDad7Q6xuspclUpf22vmdc7IeDXYfTjH8TitVzstTRuGsUZ/tf5nfkPUrcxwrlnn9bqd79nHsjpytOeEAQBAEAQBAEAQBAeEBwsRcIDkPH/AGYvjdJivCsRtq6WgZp6mP8A6/ToqZ477HR02sa92ZzZkoYxnfAtILmuBBBDhyIWq4tM9BjyR2Rk/mWJavTwixPMrKiVTy3yTnss4HfjlUzGMVhIwyF94Y3j+svH/AfmdOq2McPJydXqtq2R7ndQANleck9QBAEAQBAEAQBAEAQBAeEXQEb4h4F4e4gmNRX0OWpO88DzG93rb5ve6i4Jl2PUZMapPg1uH9lfCtHO2Z9LNVFpuG1Mxc33aLA+91jZEnLV5WqsmscbImNjja1jGizWtFgB0AUzWKkAQBAEAQBAEAQBAEAQBAEAQBAEAQBAEAQBAEB//9k='
                 , height="130px" ,style={'border-radius':'80px',
                                'textAlign':'left','margin-left':'20px',
                                'margin-bottom':'20px',
                                'border': '1px solid wheat'
                                ,'box-shadow':'20px'}

                       ),
                ], style={'border-radius':'60px',

                                'textAlign':'left','margin-left':'20px','display':'inline-block'},),
              html.Div([

                  html.P("Train/Test Split"),
                  html.H5("{0}/{1}".format(70,30)),

               ],style={'background-color':'wheat','width':'200px','height':'80px','margin-left':'70px'
                        ,'text-align':'center','border-radius':'15px 50px','display':'inline-block',
                         'cursor':'pointer'}),

                html.Div([

                  html.P("Training Accuracy"),
                  html.H5("{0:.2f}%".format(0.859*100)),

               ],style={'background-color':'wheat','width':'200px','height':'80px','margin-left':'10px'
                        ,'text-align':'center','border-radius':'15px 50px','display':'inline-block',
                        'cursor':'pointer'}),

              html.Div([
                  html.P("Testing Accuracy"),
                  html.H5("{0:.2f}%".format(0.884*100)),

               ],style={'background-color':'wheat','width':'200px','height':'80px','margin-left':'10px'
                        ,'text-align':'center','border-radius':'15px 50px','display':'inline-block',
                        'cursor':'pointer'}),


               html.Div([
                         daq.LEDDisplay(
                            value=df.shape[0],
                            label = "Records",
                            size=20,
                            color = 'white',
                            backgroundColor='orange',
                            style={'display':'inline-block','border-radius':'50px 50px','margin-left':'70px'} #200px

                        ),
              ],style={'display':'inline-block'}) ,

              html.Div([
                         daq.LEDDisplay(
                            value=XX_train.shape[0],
                            label = "Train",
                            size=20,
                            color = 'white',
                            backgroundColor='orange',
                            style={'display':'inline-block','border-radius':'50px 50px','margin-left':'20px'}

                        ),
              ],style={'display':'inline-block'}) ,

              html.Div(
                  [

                    daq.LEDDisplay(
                            value=XX_test.shape[0],
                            label = "Test",
                            size=20,
                            color = 'white',
                            backgroundColor='orange',
                             style={'display':'inline-block','border-radius':'50px 50px','margin-left':'20px'}
                        ),

                  ],style={'display':'inline-block'}),
            ]),

      #########################################gauge1###################################################

    html.Br(),
    html.Br(),
    html.Center('Select Data Row To Predict',style={'display':'inline-block','margin-left':'20px','width':'52%','border-radius':'10px','height':'30px','background-color':'azure'}),

    html.Div([
         #row1
         dbc.Row(
            [
                dbc.Col(html.H6("Sex"),style={'color':'white'}),
                dbc.Col(html.H6("Resting ECG"),style={'color':'white'}),
                dbc.Col(html.H6("Chest Pain Type"),style={'color':'white'}),
            ],
            align="start",
        ),

        #row1
        dbc.Row([
               #col1
                dbc.Col(
                  dcc.Dropdown(id='sex_dd',
                   options=[
                       {'label':'Female' ,'value':'Female'},
                       {'label':'Male' ,'value':'Male'}
                   ],
                    value='Female',
                    style={'color':'black'}

                    )

                ),
            #col2
            dbc.Col(
                  dcc.Dropdown(id='restingECG_dd',
                   options=[
                       {'label':'Normal' ,'value':'Normal'},
                       {'label':'ST' ,'value':'ST'},
                       {'label':'LVH' ,'value':'LVH'},
                   ],

                    value='LVH'   ,
                    style={'color':'black','border': '1px solid moss-rose'}

                    )

                ) ,
             #col3
               dbc.Col(
                  dcc.Dropdown(id='chestPainType_dd',
                   options=[
                       {'label':'ATA' ,'value':'ATA'},
                       {'label':'NAP' ,'value':'NAP'},
                       {'label':'ASY' ,'value':'ASY'},
                       {'label':'TA' ,'value':'TA'},
                   ],

                    value='TA' ,
                    style={'color':'black'}

                    )
                ),
                ],

            align="center",
            style={'margin-bottom':'3%'},
        ),

        #row2
         dbc.Row(
            [
                dbc.Col(html.H6("Exercise Angina"),style={'color':'white'}),
                dbc.Col(html.H6("ST Slope"),style={'color':'white'}),
                dbc.Col(html.H6("Age"),style={'color':'white'}),
            ],
            align="start",
        ),
        #row2
         dbc.Row([
               #col1
                dbc.Col(
                  dcc.Dropdown(id='ExerciseAngina_dd',
                   options=[
                       {'label':'Yes' ,'value':'Yes'},
                       {'label':'No' ,'value':'No'}
                   ],
                    value='No',
                    style={'width':'200px','color':'black',}

                    )

                ),
            #col2
            dbc.Col(
                  dcc.Dropdown(id='STslope_dd',
                   options=[
                       {'label':'Flat' ,'value':'Flat'},
                       {'label':'Up' ,'value':'Up'},
                       {'label':'Down' ,'value':'Down'},
                   ],

                    value='Up' ,
                    style={'width':'200px','color':'black','border': '0px solid #e377c2'}

                    )

                ) ,
             #col3
               dbc.Col(
                  daq.NumericInput(
                            id='id-Age',
                            min=0,
                            max=100,
                            size = 75,
                            value=50,

                        ),
                    style={'color':'black', 'padding-left':'0%'}

                    )
                ],
            align="center",
             style={'margin-bottom':'3%'},
        ),
        dbc.Row(
            [
                dbc.Col(html.H6("Resting Blood Pressure"),style={'color':'white'}),
                dbc.Col(html.H6("Cholesterole"),style={'color':'white'}),
                dbc.Col(html.H6("Fasting Blood Sugare"),style={'color':'white'}),

            ],
            align="center",
        ),

        dbc.Row(
            [
               #col1
                dbc.Col(
                   daq.NumericInput(
                            id='id_ResBlood_Pres',
                            min=0,
                            max=200,
                            size = 75,
                            value=100,

                        ),
                    style={'color':'black'}

                ),
            #col2
            dbc.Col(
                   daq.NumericInput(
                            id='id_cholesterole',
                            min=0,
                            max=603,
                            size = 75,
                            value=180,

                        ),
                    style={'color':'black'}


                ) ,
             #col3
               dbc.Col(
                  dcc.Dropdown(
                            id= 'id_Fasting_Blood_Sugar',
                            options=[
                            {'label': 'more than 120 mg/dl','value': 'more than 120 mg/dl'},
                            {'label':'less than 120 mg/dl', 'value':'less than 120 mg/dl'},
                               ],
                               value=['less than 120 mg/dl'],
                        ), #end drop down


                    style={'color':'black'}
                ) ,

            ],
             align="start",
            style={'margin-bottom':'3%'},
        ),

          #row1
         dbc.Row(
            [
                dbc.Col(html.H6("Max Heart Rate"),style={'color':'white'}),
                dbc.Col(html.H6("Old Peak"),style={'color':'white'}),
                dbc.Col(html.Div(
                    style={'color':'white'})
                       ),
            ],
            align="start",
        ),
         dbc.Row(
            [
               #col1
                dbc.Col(
                   daq.NumericInput(
                            id='id_MHR',
                            min=100,
                            max=205,
                            size = 75,
                            value=140,

                        ),
                    style={'color':'black'}

                ),
            #col2
            dbc.Col(
                     daq.NumericInput(
                            id='id_oldpeak',
                            size = 75,
                            value=2.5,

                        ),
                    style={'color':'black'}

                ) ,
             #col3
               dbc.Col(
                   [

                        dbc.Button("PREDICT!",id='id_PREDICT', size="lg",className="gap-2 col-10 mx-auto",n_clicks=0,
                                   style={'background-color':'azure','color':'black'}),


                   ]),
            ],
             align="start",
             style={'margin-bottom':'3%'},
        ),
        ],className="pad-row" ,

        style={'display':'inline-block','background-color':'silver','width':'52%','padding':'20px','border-radius':'10px','margin-left':'20px'}

        ),

     html.Div([
               dcc.Graph(figure=fig,style={'background-color':'#7f7f7f'}),
     ],style={'display':'inline-block','width':'500px','margin-right':'50px','float':'right','marginLeft': 'auto'
     ,'margin-bottom':'60px','box-shadow':'darkgray 0px 0px 10px 0px'})   ,

     html.Br(),
     html.Br(),
     html.Div([

       html.H5(id='predict_res_id',style={'margin-left':'20px','color':'greenish'})

     ], style={'padding':'10px','align':'center','background-color':'silver','border-radius':'30px','height':'120px','width':'52%','margin-left':'20px'})
     ,
      html.Br(),

      ]) #tab1
      ,

      ##############################################SECOND TAB###############################################

       dcc.Tab(label='Heart disease failure data Dashboard',style={'background-color':'azure' }, children=[

              dbc.Container( [

        dbc.Row(
            [
                dbc.Col(
                    html.Img(
                   src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAHwAfAMBEQACEQEDEQH/xAAcAAEAAQUBAQAAAAAAAAAAAAAABgIDBAUHAQj/xAA7EAABAwIEAwYEBAILAAAAAAABAAIDBBEFEiExBkFRBxMiYXGBFDKRwVKhsdEjMxU0QlNicoKywtLw/8QAGgEBAAIDAQAAAAAAAAAAAAAAAAIDAQQFBv/EAC4RAAICAQMDAQcDBQAAAAAAAAABAhEDBBIhBTFBURMiMmFxgcFCodEUIySR8P/aAAwDAQACEQMRAD8A7igCAIAgCAIDwuA3QGNW4jR0DWuramGna75TLIG39LrEpKK5ZZjw5MrrHFv6F2OeKSnbURyMdC5ucSB3hLd736JfFkXGSlta5Kaerp6kF1PNFKBuY3hwH0RNPsJQlD4lRXJNHE3NK9rG9XGwWTCTk6Ssqa4PALSCCLgg7oY7FSAIAgCAIAgCAIAgCAgnaDE74+kcXvyOjNm5jYEHe23MfRaepdNHf6NteOXHKZBsZnrcwiq5pJKVr/4edxIbcW06cvotdttcna00IrI3FU6/JNp8TZh3ZpTRZwaippvh4mA63NwfoL/RbTntwL1OJHTvN1WTriLt/bn9zWcJYtTYBgWKVmVhnfO2GGEGxkcGC3t4jdQxzUItm31HTT1epx4/FNt+nJYw3CMV4vnmqqisYRG7KZpmFzWu6RsuBpfr9ViMJZbbZbn1On6bFQjHv4XD+7Oj4Bhr8IwuGhfUGoMQIEhbl0JJtbkBstuEdsaPM6rP/UZpZKqzYqZrhAEAQBAEAQBAEAQEI47eH11JF/dxOe73It/tWlqu6R3ujprHOXq0c14grfiJhSxeINN3+Z6LVPTabHtW5lVEx7Iw6cNDrWzEaqceO5HI03UT3+D3+eOZjX+aUvBm5baa4Om9mNRF/QL6QuHfwTOLh1DjcH9R7Lc0z92jyvXIS/qfaeGl+xMVsHGCAIAgCAIAgCAIAgKXvbG0ue4BoFyTyQLng5Rxlib3VtVUNvd5DYdOWw+5XOzNudnrul4oPFGK+5G6OiZTxGWfxSO1uVFRrlnTyZXJ7Y9jClknr6ptNSgZibeJwaB6k6AeaJX3JuUcMdzR5iGDVtBG6aYQyRBwaZYX5gCduQ6HVZoji1ccklFqm/Uy+E8bmwXHaSpEh7kvEc7b/Mx2h+m/spQeyVkOoaWOp08oee6+q/nsfQC6B4AIAgCAIAgCAIAgMKtxKGk8Ju+XlG3f36KMpqJZDFKfY0VbWVFX/NIazkxuw/dUSk2bkMUYdiEcU2kroBuGku9wLfdUz8HZ6Y6c3/3c0mJVGSH7KuTs6+GHNlnhqPvDPNYm5ADrctSfzSXCI55XI3lfQ/0jTCKXMGjUFptYqCdFClGLvyamPhvuqhjp5nSQNcC+NoAc8cxfldT3oteebi1F8nXsI4qw3EY2jM+lkOnd1Ayn67LejmhLizyObp2oxc1a+X8dzetIcAQbg6q00T1AEAQBAEAQGHilYKOlLxbO45WA9SoylSLMcN8qIznNy9xzPcblx3JWvZ0EklSMSrrGRNN3XPkomSKYm91ZOBF8zXXc4bN0tbzVc6fY6vT4vG3KfnsvyURYbC9wMrBK4c36ge2yps6MsjSNtDDHG0WAQ1JTkzx04vliBef8OykotlcnGHxuig09VLqY/ZS9n8yC12OLpJmPXGWGlmaYckndkNIvdR2tPkvjkjljcX9ifcANmbwtRCfNfxZc2+XMbLfw/Aed6i4vUyr5EjVpohAEAQBAEBGeNKgQtpA4OLXF506i37qnL4NrTcNkOxDGW0sV7uudgTqVQ+FydHDhnmltgiH12O1dRKWhwYwXuBzt5qtybR2MOjxYnb5fzKcHxE087xP3j2PGuXXKetlFx4o2Gk3fkklLXQSi8Esb+rb6/TdQpohKNluTFaT4uOkfO0SPNg0a69CdgrMcL5Zqamcscfc7kloBAB4ALEaE/dXHHu+TZNLQFkGj4tjMmDVBicWPaM92mxsNVhm1opbdRG/Wv9m07KcWqsRwyqir60zywSgRse6744y0WueYJB3V2GTa5Zr9ZwRxZk4RpNfaydK45AQBAEAQBAariTDjiWGPjjF5mHPH5kcvcKMlaLMU9srOCV9d8bXvnDrxN+TXTKNlz5u+T2elxLDBR+7+phMD7ZmtJI1KeCwz8NDY7yEXB0Cw2ZLGJENd3lgLg7LKIyltVmHTU5mcXv8Ak29SpSe3hFGPF7R7pdjZMxmvoqjPRy+EDK4P8QdbrdIv1I6jBGaSUeES7AeLIa8iGcfD1PJpN2O9D9lNM5mXTuC3LlG8kYKxhik8bZBksDvdZKFLb7y8Eo4Y4Ww/hplQ2g75zp3AvkmcHOsNhoBoLrZhBQ7Gnqtbl1Uk8nj0N6pmoEAQBAEAQGu4kqDScPYnUtveGkleLeTCVGbqLZbp47ssYvy0fOMQHw7rb3Fh1A/8FzezPbRe6LkbGmex0Qa217ahZZkrjjbE0ht7brBgwakipe1oIFzYNsb/AKKStEZpS4ZdqAIDAyM3azxHoTdYXJMxpvA4huoOoPks2Yboxy7K8G9jdSXKNaT2zs6n2VTx4vPN8ZI41VHle1vJ4Ozj5gj9Fdge7ucXqeL2NOHZnUhsto44QBAEAQBAEBgY9SmtwPEKRou6emkjHqWkKMlaaJ4pbZqXoz5pikzRtcNiLj3C5x7KE/BkN2CkXlRc4ixc4j1SgUiTumueD4vlaehKwRlJRW4Rk/DWcdWv09CD+yeeDEG65KHG+lvMIyT9CwRmcQeQUk6RqSjum7J72Md4eJ6k3OUUT83mc7LfdW4fj4Ob1OX+Mk+9/hnaltnBCAIAgCAIAgBQHzhxdhbsE4mxCgylsQlMsGmhjebtt6at9lo5Y7ZHptHm9piT8o10btLc1BHThK0XBYi5vvaw5oTLM5/lAAAanRI9mU5v0ovNyiAh2neOuD0tzUWWLmP1LLw5rrO0IQi2138Fs/Nfy1UvBrSdys6n2H4eTT4lizgQyVzaeK4/DcuI8ruaP9K2sEatnE6nkTlGK8HU1ecsIAgCAIAgCAIDnva7w1LimGR4tQRmSsoAc8bRrJCfmA8xv9VVlhuRvaHUeynT7M43FI17Q9huDstOqPRQlzaLveAC6Jcl7yIoddzYyTrqEvuVyuSi2XZXWeW/hGUKJc/iotlxdG0nceH9lmuSuTuFlqCCfEa2DD6BveVNQ8RsaOZP26+SnGNs08uSMItvwfS3DmDw4DgtJhlNqyCPKXfjdu53ublb0VSo8zlyPJNyfk2SyQCAIAgCAIAgCAEIDivadwJLhc8uO4FDeiec9VTsH8k83gfh69PTajJj8o62j1n6JHPYphLYD5ugWvVHV3WZF9AoGyuyPZSTI71RGcknudFiolDad7Q6xuspclUpf22vmdc7IeDXYfTjH8TitVzstTRuGsUZ/tf5nfkPUrcxwrlnn9bqd79nHsjpytOeEAQBAEAQBAEAQBAeEBwsRcIDkPH/AGYvjdJivCsRtq6WgZp6mP8A6/ToqZ477HR02sa92ZzZkoYxnfAtILmuBBBDhyIWq4tM9BjyR2Rk/mWJavTwixPMrKiVTy3yTnss4HfjlUzGMVhIwyF94Y3j+svH/AfmdOq2McPJydXqtq2R7ndQANleck9QBAEAQBAEAQBAEAQBAeEXQEb4h4F4e4gmNRX0OWpO88DzG93rb5ve6i4Jl2PUZMapPg1uH9lfCtHO2Z9LNVFpuG1Mxc33aLA+91jZEnLV5WqsmscbImNjja1jGizWtFgB0AUzWKkAQBAEAQBAEAQBAEAQBAEAQBAEAQBAEAQBAEB//9k='
                 , height="130px" ,style={'border-radius':'80px',
                                'textAlign':'left','margin-left':'20px',
                                'margin-bottom':'20px',
                                'border': '1px solid wheat'
                                ,'box-shadow':'20px'}

                       ), ),

            dbc.Col(
                drop_down_sex
            , style={'width' : '33.33%'}
            ),

            dbc.Col(
                slider ,
            )
            ]
       ,style={'margin-bottom':'-1px'} ),#end row

    ###################
    ## second row ##
    ###################

        dbc.Row([
            dbc.Col(
                  html.Div(
                 card_man,
                ) , #div
                width= 'auto' ,
            ) ,#col

            dbc.Col(
                html.Div(
                card_woman,
                ),
                width = 'auto',
            ),

            dbc.Col(
                html.Div(
                 card_patient,
                ) ,#div
                width='auto' ,
            ) , #col

            dbc.Col(
                 html.Div(
                 card_non_patient,
                ), #div ,
                width = 'auto',
            ),#col

            ] ,
            style ={ 'margin-top':'-20px' , 'margin-left':'45px'}

        ), #end row
    #################
    ### third row ###
    #################
    dbc.Row([
        dbc.Col([
            dcc.Graph(id= 'figure_1_col_1'  , style = {'background-color':'lightgray','box-shadow':'0px 0px 10px 0px darkgray'}) ,

            dcc.Graph(id= 'figure_2_col_1'   , style = {'background-color':'lightgray','margin-top':'10px','height':'50%' , 'box-shadow':'0px 0px 10px 0px darkgray'})

        ]
        ,width = 'auto'
        , style = {}),

        dbc.Col(
             dcc.Loading(dcc.Graph(id= 'figure_1_col_2'  ,
             style={'box-shadow':'0px 0px 10px 0px darkgray' , 'margin-top':'20px' , 'margin-left':'52px'},), type="cube"), #top right bottom left
            width = 'auto' ,

        )


    ] , style = {'margin-top':'20px'}
    ) , #row

    ######################
    ## fourth row ########
    ######################

    dbc.Row(
    [] , style ={'margin-top':'20px'}
    )
    ], style={"height": "100%" ,'background-color': background_color , 'max-width':'100%'},)#end container

       ]),

      ]) #parent tab

    ]) #layout dev


############################
## call back for the cards ##
#############################
mapping_dict = {'Male' :"M" , 'Female' : "F"}
@app.callback(
    Output(component_id= 'number_of_men', component_property='children'),
    Output(component_id= 'number_of_women', component_property='children'),
    Output(component_id= 'number_of_patients', component_property='children'),
    Output(component_id= 'number_of_non_patients', component_property='children'),
    Output(component_id= 'figure_1_col_1', component_property='figure'),
    Output(component_id= 'figure_2_col_1', component_property='figure'),
    Output(component_id= 'figure_1_col_2', component_property='figure'),

    Input(component_id= 'age_range', component_property='value'),
    Input(component_id= 'drop_down_sex', component_property='value'),
)


def update_My_div(age_range , drop_down_values):
    df_temp = df2[ (df2['Age'] <= max(age_range) )& (df2['Age']>= min(age_range) )]
    if len(drop_down_values) == 1 :
        df_temp = df_temp[df_temp['Sex'] == mapping_dict[drop_down_values[0] ] ]

    number_of_men = df_temp[df_temp['Sex'] == "M"].count()[0]
    number_of_women = df_temp[df_temp['Sex'] == "F"].count()[0]
    number_of_patients = df_temp[df_temp['HeartDisease'] == 'yes'].count()[0]
    number_of_non_patients = df_temp[df_temp['HeartDisease'] == 'no'].count()[0]

    figure_chest_pain_age = px.box(df_temp , x = 'ChestPainType',y='Age',title= 'which chest pain is critical ?' , color = 'HeartDisease', category_orders= {'HeartDisease' : ['no' ,'yes']})
    figure_box_plot_age = px.box(df_temp ,
                             y = 'Age' ,
                             x = 'HeartDisease' ,
                             color = 'HeartDisease',
                                 title='.             Age v/s HeartDisease',
                             width=400,
                             height=250,
                            )

    figure_sex_patient =  px.histogram( df_temp ,
                                     color = 'HeartDisease',
                                     x = 'Sex' ,
                                     template='plotly' ,
                                     barmode = 'group' ,
                                     title = '.             Gender v/s HeartDisease' ,
                                       barnorm='percent' ,
                                       text_auto='.2f',
                                    )
    return number_of_men , number_of_women , number_of_patients ,number_of_non_patients ,figure_box_plot_age , figure_sex_patient ,figure_chest_pain_age

@app.callback(
    Output('predict_res_id', 'children'),
    [Input('id_PREDICT', 'n_clicks')],
    [State('id-Age', 'value'),
     State('sex_dd', 'value'),
     State('chestPainType_dd', 'value'),
     State('id_ResBlood_Pres', 'value'),
     State('id_cholesterole', 'value'),
     State('id_Fasting_Blood_Sugar', 'value'),
     State('restingECG_dd', 'value'),
     State('id_MHR', 'value'),
     State('ExerciseAngina_dd', 'value'),
     State('id_oldpeak', 'value'),
     State('STslope_dd', 'value')]
)

def update_output(n_clicks, age, sex, chest_pain_type, resting_bp, cholesterol, fasting_bs, resting_ecg, max_hr, exercise_angina, oldpeak, st_slope):
    if n_clicks > 0:
        result = make_prediction(1, age, sex, chest_pain_type, resting_bp, cholesterol, fasting_bs, resting_ecg, max_hr, exercise_angina, oldpeak, st_slope)
        return html.Div([
            html.H2(result[0]),
            html.H2(result[1])
        ])
if __name__ == '__main__':
    app.run_server(debug=True, port=8060)

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
# prompt: i need full code for deploy it in python anywhere

1. **Create a PythonAnywhere account and domain:**
    - Go to https://www.pythonanywhere.com/ and sign up for an account.
    - Once you have created an account, click on "Web" in the top menu and then click on "Add a New Web App".
    - Select "Python" as the web framework and enter a name for your web app.
    - Click on "Create Web App" and your web app will be created.

2. **Install the necessary libraries:**
    - In your PythonAnywhere console, run the following command to install the necessary libraries:
